In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('$5000')

['$', '5000']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['how','are','fwafaw','you'],['you','no','no','bro','how'])

array([1., 0., 0., 0., 1.], dtype=float32)

In [8]:
data = pd.read_csv('./tweets.csv').dropna()

In [9]:
X = data['text']
y = data['target']

In [10]:
all_words = []
all_data = []
tags = []

In [11]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

7952it [00:02, 2793.01it/s]


In [12]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [13]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [14]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [15]:
X = []
y = []

In [16]:
for sentence,tag in tqdm(all_data):
    bag = bag_of_words(sentence,all_words)
    X.append(bag)
    y.append(tags.index(tag))

100%|██████████████████████████████████████| 7952/7952 [00:28<00:00, 274.51it/s]


In [17]:
X[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [18]:
y[0]

1

In [19]:
from sklearn.model_selection import *

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [21]:
device = 'cuda:0'

In [22]:
# print('-')
# X_train = torch.from_numpy(np.array(X_train)).to(device).float()
# print('-')
# X_test = torch.from_numpy(np.array(X_test)).to(device).float()
# print('-')
# y_train = torch.from_numpy(np.array(y_train)).to(device)
# print('-')
# y_test = torch.from_numpy(np.array(y_test)).to(device)
# print('-')

In [23]:
# torch.save(X_train,'X_train.pth')
# torch.save(X_test,'X_test.pth')
# torch.save(y_train,'y_train.pth')
# torch.save(y_test,'y_test.pth')
# torch.save(all_data,'all_data.pth')
# torch.save(all_words,'all_words.pth')
# torch.save(tags,'tags.pth')
X_train = torch.load('X_train.pth').to(device).float()
y_train = torch.load('y_train.pth').to(device).float()
X_test = torch.load('X_test.pth').to(device).float()
y_test = torch.load('y_test.pth').to(device).float()
all_data = list(torch.load('all_data.pth'))
all_words = list(torch.load('all_words.pth'))
tags = list(torch.load('tags.pth'))

In [24]:
class Model(Module):
    def __init__(self,hidden=1024,iters=20):
        super().__init__()
        self.activtion = ReLU()
        self.hidden = hidden
        self.iters = iters
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden)
        self.batchnorm = BatchNorm1d(hidden)
        self.linear3 = Linear(hidden,1)
        self.linear3activation = Sigmoid()
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.batchnorm(self.activtion(self.linear2(preds)))
        preds = self.linear3activation(self.linear3(preds))
        return preds

In [25]:
model = Model().to(device)

In [26]:
criterion = MSELoss()

In [27]:
optimizer = Adam(model.parameters(),lr=0.001)

In [28]:
epochs = 1000

In [29]:
batch_size = 32

In [30]:
def get_accuracy(model,X,y):
    total = 0
    correct = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(torch.round(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [31]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds.view(-1,1),y.view(-1,1))
    return loss.item()

In [32]:
PROJECT_NAME = 'Disaster-Tweets'

In [33]:
import wandb

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


 11%|████▌                                     | 11/100 [01:00<08:09,  5.50s/it]